# Trying out if we can use the street model to detect cars
In this approach, we thought about using both models to detect cars. First, we use the street model to detect streets and then we use the car model to detect cars on the streets. We run images through the street model and then we black out the non-street areas. We then use the car model to detect cars on the blacked out images. We thought that this approach might be better than using the car model directly on the aerial images because there is less noise in the images and therefore less confusing information for the model. Unfortunately we did not go through with this approach because the results were not promising (worse dice score e.g. 0,63 vs 0,78). We think that this may be because the street model is not good in detecting parking lots. Since many Cars in the aerial images are parked on parking lots, the car model does not detect them when the pixels are blacked out.

In [1]:
from fastai.vision.all import *
import numpy as np
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# check if cuda is available
print(torch.cuda.is_available())

## Load Street Trainingsdata

In [ ]:
# define a function to get the street images
def get_street_images(name):
    return get_image_files("/kaggle/input/tds-streets/train_test_split")



In [ ]:
for i in get_street_images("x"): 
    print(i)
    break

# Load Street Model

In [ ]:
def mask_function(name):
    return np.load(str(name).replace("img.png", "label_street_mask.npy"))
satellite_block = DataBlock(blocks = (ImageBlock, MaskBlock(codes = ["background","street"])),
                 get_items=get_street_images,
                 splitter=GrandparentSplitter(train_name='train', valid_name='test'),
                 get_y=mask_function,
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=256, max_lighting=0.3)
)

dataloader = satellite_block.dataloaders("/kaggle/input/tds-streets/train_test_split", bs=1)
opt_func = Adam
learner = unet_learner(dataloader,resnet34,metrics=Dice,opt_func=opt_func)

path = Path("/kaggle/input/streetmodel") 
learner.load(path/"best_model")


In [ ]:
from PIL import Image
import numpy as np

def paint_non_street_black(path, street_array):
    # Load the original image
    original_image = Image.open(path)  # Replace with your image path

    # Convert the original image to a numpy array
    original_array = np.array(original_image)
    if len(original_array.shape) == 3:
        mask_array = np.stack([street_array]*3, axis=-1)

    # Modify the original image: set pixels to black where mask_array is 0
    modified_array = np.where( mask_array== 0, 0, original_array)

    # Convert the modified numpy array back to an image
    modified_image = Image.fromarray(modified_array.astype('uint8'))
    return modified_image

In [ ]:
!rm -r /kaggle/working

In [ ]:

# Perform for each image the prediction on the street model. We then get the np array with detected streets=1 and not-detected-streets=0
for i in get_street_images(""):
    prediction_street = learner.predict(i)[0].numpy()
    # scale it up to 1000 * 1000
    # Convert the numpy array to a PIL Image
    img = Image.fromarray(np.uint8(prediction_street * 255))  # Scale and convert to uint8

    # Resize the image to 1000x1000
    resized_img = img.resize((1000, 1000))
    resized_array = np.array(resized_img)
    result_image= paint_non_street_black(i,resized_array)

    # Using regex to extract the pattern where 'validation' can be any string
    extracted = re.search(r'/([^/]+)/[\d_]+/', str(i))
    extracted = extracted.group(0)[1:-1] if extracted else None
    os.makedirs(f"/kaggle/working/streets_blacked/train_test_split/{extracted}/")
    result_image.save(f'/kaggle/working/streets_blacked/train_test_split/{extracted}/img.png') 


In [ ]:
"25961_12100" in os.listdir("/kaggle/working/streets_blacked/train_test_split/test/")

In [ ]:
from PIL import Image
import numpy as np

def manipulate_mask(path_to_blacked):
    if not str(path_to_blacked).endswith("/"): 
        path_to_blacked=str(path_to_blacked)+"/"
    # Load the image
    image = Image.open(str(path_to_blacked)+"img.png")
    # Convert the image to a numpy array
    image_array = np.array(image)
    # load mask 
    target_path= path_to_blacked.replace("working/streets_blacked","input/car-segmentation")+"label.npy"
    
    # we had to do this because the path was not always correct and so we had to search fo the correct image
    if os.path.isfile(target_path):
        pass
    elif os.path.exists(target_path.replace("/train/","/test/")):
        target_path=target_path.replace("/train/","/test/")
        
    elif os.path.exists(target_path.replace("/train/","/validation/")):
        target_path=target_path.replace("/train/","/validation/")
        
    elif os.path.exists(target_path.replace("/test/","/train/")):
        target_path=target_path.replace("/test/","/train/")
        
    elif os.path.exists(target_path.replace("/test/","/validation/")):
        target_path=target_path.replace("/test/","/validation/")
        
    elif os.path.exists(target_path.replace("/validation/","/train/")):
        target_path=target_path.replace("/validation/","/train/")
        
    elif os.path.exists(target_path.replace("/validation/","/test/")):
        target_path=target_path.replace("/validation/","/test/")
    mask_array= np.load(target_path)
    # Check if the PNG image is in grayscale or color
    if len(image_array.shape) == 3:
        # Assuming color image, identify black pixels (R=G=B=0)
        black_pixels_mask = np.all(image_array == [0, 0, 0], axis=-1)
    else:
        # Assuming grayscale image, black pixels are 0
        black_pixels_mask = (image_array == 0)

    # Set corresponding positions in npy_array to 0 where black_pixels_mask is True
    mask_array[black_pixels_mask] = 0
    return mask_array
    

In [ ]:
modified_array=manipulate_mask("/kaggle/working/streets_blacked/train_test_split/test/26794_12530/")
#modified_array= np.load("/kaggle/input/car-segmentation/train_test_split/test/26794_12530/label.npy")
modified_image = Image.fromarray(modified_array.astype('uint8'))

# Display the modified image (using matplotlib)
import matplotlib.pyplot as plt
plt.imshow(modified_image)
plt.axis('off')
plt.show()

In [ ]:
 image = Image.open("/kaggle/working/streets_blacked/train_test_split/test/26794_12530/"+"img.png")
    # Convert the image to a numpy array
image_array = np.array(image)
import matplotlib.pyplot as plt
plt.imshow(image_array)
plt.axis('off')
plt.show()

## Load the Car Model

In [ ]:
def get_images(name):
    return get_image_files("/kaggle/working/streets_blacked/train_test_split")
# define a function to get the numpy mask for the given path
def get_mask(path):
    return manipulate_mask(path.parent)


aerial_Block = DataBlock(blocks = (ImageBlock, MaskBlock(codes = ["nothing","car"])),
                            splitter=GrandparentSplitter(train_name='train', valid_name='test'),
                            get_items=get_images,
                            get_y=get_mask,
                            batch_tfms=aug_transforms(size=500, max_lighting=0.3))
dataloader = aerial_Block.dataloaders("/kaggle/working/streets_blacked/train_test_split",bs=8)
torch.cuda.empty_cache()


In [ ]:
dataloader.show_batch(max_n=4,figsize=(10,10))


In [ ]:
learner = unet_learner(dataloader,resnet34,metrics=Dice)

In [ ]:
lr = learner.lr_find()

In [ ]:
learner.fit_one_cycle(20, lr_max=lr.valley, cbs=[EarlyStoppingCallback(monitor='dice', min_delta=0.01, patience=5, comp=np.greater),
                                           SaveModelCallback(monitor='dice', min_delta=0.01, comp=np.greater, fname='best_frozen_model')])

## unfreeze and retrain

In [ ]:
learner = learner.load('best_frozen_model')
learner.unfreeze()

In [ ]:
lr = learner.lr_find()

In [ ]:
learner.fit_one_cycle(
    20, lr_max=slice(lr.valley/10,lr.valley*10),
    cbs=[EarlyStoppingCallback(monitor='dice', min_delta=0.005, patience=5, comp=np.greater),
         SaveModelCallback(monitor='dice', comp=np.greater, fname='best_model')]
         )